# 라이브러리

In [1]:
import pandas as pd
import random
import os
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# 한글 폰트 깨짐 현상 해결을 위한 나눔 폰트 설치
# 코드 1회 실행 후 주석 처리하고 런타임 재시작 및 모두 실행
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

## 데이터 로드

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
typeC_data = pd.read_csv('/content/drive/MyDrive/Project/typeC_TrafficData.csv')

In [6]:
typeC_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43840 entries, 0 to 43839
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datetime       43840 non-null  object 
 1   dow            43840 non-null  int64  
 2   branch_name    43840 non-null  object 
 3   district_name  43840 non-null  object 
 4   lane           43840 non-null  int64  
 5   branch_num     43840 non-null  object 
 6   dep_point      43840 non-null  object 
 7   arr_point      43840 non-null  object 
 8   time_0         41724 non-null  float64
 9   time_1         41745 non-null  float64
 10  time_2         41797 non-null  float64
 11  time_3         41828 non-null  float64
 12  time_4         41858 non-null  float64
 13  time_5         41861 non-null  float64
 14  time_6         41864 non-null  float64
 15  time_7         41882 non-null  float64
 16  time_8         41853 non-null  float64
 17  time_9         41838 non-null  float64
 18  time_1

In [7]:
typeC_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43840 entries, 0 to 43839
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datetime       43840 non-null  object 
 1   dow            43840 non-null  int64  
 2   branch_name    43840 non-null  object 
 3   district_name  43840 non-null  object 
 4   lane           43840 non-null  int64  
 5   branch_num     43840 non-null  object 
 6   dep_point      43840 non-null  object 
 7   arr_point      43840 non-null  object 
 8   time_0         41724 non-null  float64
 9   time_1         41745 non-null  float64
 10  time_2         41797 non-null  float64
 11  time_3         41828 non-null  float64
 12  time_4         41858 non-null  float64
 13  time_5         41861 non-null  float64
 14  time_6         41864 non-null  float64
 15  time_7         41882 non-null  float64
 16  time_8         41853 non-null  float64
 17  time_9         41838 non-null  float64
 18  time_1

In [8]:
typeC_data.isnull().sum()

datetime            0
dow                 0
branch_name         0
district_name       0
lane                0
branch_num          0
dep_point           0
arr_point           0
time_0           2116
time_1           2095
time_2           2043
time_3           2012
time_4           1982
time_5           1979
time_6           1976
time_7           1958
time_8           1987
time_9           2002
time_10          1987
time_11          1997
time_12          1999
time_13          1991
time_14          1980
time_15          1950
time_16          1918
time_17          1895
time_18          1892
time_19          1874
time_20          1881
time_21          1881
time_22          2082
time_23          2240
dtype: int64

# 출발지점 리스트로 추출

In [9]:
typeC_bn_dp = typeC_data[['branch_num', 'dep_point']]                
typeC_bn_dp = typeC_bn_dp .drop_duplicates(['dep_point'], ignore_index = True)

new_row = pd.DataFrame([['C-12', '반포대교남단']], columns = typeC_bn_dp.columns)
typeC_bn_dp = pd.concat([typeC_bn_dp.iloc[:21], new_row, typeC_bn_dp.iloc[21:]], ignore_index = True)

typeC_dp_list = typeC_bn_dp['dep_point'].tolist()
typeC_dp_list

['행주대교남단',
 '행주대교북단',
 '가양대교남단',
 '가양대교북단',
 '성산대교남단',
 '성산대교북단',
 '양화대교남단',
 '양화대교북단',
 '서강대교남단',
 '서강대교북단',
 '마포대교남단',
 '마포대교북단',
 '원효대교남단',
 '원효대교북단',
 '한강대교남단',
 '한강대교북단',
 '동작대교남단',
 '동작대교북단',
 '반포대교남단',
 '반포대교북단',
 '반포고가차도',
 '반포대교남단',
 '한남대교남단',
 '한남대교북단',
 '동호대교남단',
 '동호대교북단',
 '성수대교남단',
 '성수대교북단',
 '영동대교남단',
 '영동대교북단',
 '청담대교남단',
 '청담대교북단',
 '잠실대교남단',
 '잠실대교북단',
 '올림픽대교남단',
 '올림픽대교북단',
 '천호대교남단',
 '천호대교북단',
 '광진교남단',
 '광진교북단']

# 출발지점 별 시간 데이터 추출

In [10]:
typeC_new_data = []
for i in range(0, len(typeC_dp_list)) : 
  typeC_data_temp = typeC_data.loc[typeC_data.dep_point == typeC_dp_list[i]]
  typeC_data_time = typeC_data_temp.iloc[:, 8:32]
  typeC_new_data.append(typeC_data_time)

In [11]:
typeC_new_data[39]

,time_0,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,time_9,...,time_14,time_15,time_16,time_17,time_18,time_19,time_20,time_21,time_22,time_23
42744,NaN,120.0,79.0,72.0,51.0,51.0,116.0,177.0,224.0,189.0,...,299.0,314.0,288.0,268.0,210.0,196.0,180.0,155.0,145.0,83.0
42745,64.0,55.0,52.0,28.0,37.0,74.0,154.0,293.0,508.0,429.0,...,354.0,391.0,410.0,484.0,498.0,363.0,264.0,217.0,164.0,101.0
42746,94.0,84.0,75.0,70.0,53.0,93.0,163.0,330.0,537.0,468.0,...,408.0,397.0,460.0,502.0,498.0,375.0,265.0,245.0,190.0,146.0
42747,96.0,93.0,90.0,93.0,50.0,91.0,159.0,315.0,468.0,446.0,...,414.0,437.0,448.0,570.0,530.0,418.0,319.0,231.0,217.0,159.0
42748,80.0,93.0,109.0,69.0,69.0,83.0,140.0,183.0,306.0,352.0,...,534.0,494.0,476.0,479.0,393.0,266.0,284.0,240.0,222.0,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43835,49.0,27.0,26.0,25.0,37.0,91.0,162.0,316.0,512.0,448.0,...,408.0,428.0,504.0,535.0,464.0,296.0,207.0,178.0,115.0,79.0
43836,63.0,32.0,28.0,21.0,41.0,90.0,178.0,288.0,561.0,490.0,...,385.0,434.0,488.0,496.0,535.0,302.0,245.0,211.0,115.0,68.0
43837,60.0,33.0,46.0,34.0,45.0,86.0,182.0,288.0,576.0,410.0,...,388.0,438.0,471.0,498.0,522.0,356.0,231.0,209.0,117.0,86.0
43838,51.0,30.0,34.0,22.0,35.0,96.0,153.0,296.0,581.0,502.0,...,463.0,558.0,653.0,625.0,606.0,385.0,281.0,214.0,129.0,82.0


# 출발지점 및 시간 별 통행량 합계

In [12]:
typeC_sum = []
for r in range(0, len(typeC_dp_list)) :
  type_data_sum = typeC_new_data[r].sum()
  typeC_sum.append(type_data_sum)

In [13]:
typeC_sum[0]

time_0      443903.0
time_1      310398.0
time_2      230010.0
time_3      231002.0
time_4      329095.0
time_5      826945.0
time_6     1800490.0
time_7     2706700.0
time_8     3112925.0
time_9     2974851.0
time_10    3019716.0
time_11    3100471.0
time_12    2931886.0
time_13    2917415.0
time_14    2988746.0
time_15    3049523.0
time_16    3313938.0
time_17    3783358.0
time_18    3822625.0
time_19    2956067.0
time_20    1897727.0
time_21    1641255.0
time_22    1353327.0
time_23     827241.0
dtype: float64

# 출발지점 별 통행량 합계

In [14]:
typeC_total = []
for e in range(0,len(typeC_dp_list)) :
  typeC_total_sum = typeC_sum[e].sum()
  typeC_total.append(typeC_total_sum)

In [15]:
print(max(typeC_total))
print(typeC_total.index(max(typeC_total)))
# 한남대교 남단 출발이 최대

98459247.0
22


# 다리 별 통행량 합계

In [16]:
typeC_bn_total = []
for o in range(0, len(typeC_dp_list)) :
  if o % 2 == 0 :
    typeC_bn_total_sum = typeC_total[o] + typeC_total[o+1]
    typeC_bn_total.append(typeC_bn_total_sum)

In [17]:
typeC_bn = typeC_data['branch_name']                
typeC_bn = typeC_bn.drop_duplicates()
typeC_bn = typeC_bn.tolist()
typeC_bn

['행주대교',
 '가양대교',
 '성산대교',
 '양화대교',
 '서강대교',
 '마포대교',
 '원효대교',
 '한강대교',
 '동작대교',
 '반포대교',
 '잠수교',
 '한남대교',
 '동호대교',
 '성수대교',
 '영동대교',
 '청담대교',
 '잠실대교',
 '올림픽대교',
 '천호대교',
 '광진교']

In [18]:
typeC_bn_total_temp = list(zip(typeC_bn_total, typeC_bn))
print(typeC_bn_total_temp)

[(100851174.0, '행주대교'), (122597928.0, '가양대교'), (149001366.0, '성산대교'), (142759604.0, '양화대교'), (45418028.0, '서강대교'), (107948634.0, '마포대교'), (49674940.0, '원효대교'), (89907701.0, '한강대교'), (57730664.0, '동작대교'), (108458017.0, '반포대교'), (78150029.0, '잠수교'), (196712869.0, '한남대교'), (75898176.0, '동호대교'), (128803740.0, '성수대교'), (128320890.0, '영동대교'), (133345367.0, '청담대교'), (99471344.0, '잠실대교'), (66286549.0, '올림픽대교'), (70572787.0, '천호대교'), (16357856.0, '광진교')]


In [20]:
typeC_bn_total_temp.sort(reverse=True)
typeC_bn_total_temp
# 한남대교 통행량이 최대

[(196712869.0, '한남대교'),
 (149001366.0, '성산대교'),
 (142759604.0, '양화대교'),
 (133345367.0, '청담대교'),
 (128803740.0, '성수대교'),
 (128320890.0, '영동대교'),
 (122597928.0, '가양대교'),
 (108458017.0, '반포대교'),
 (107948634.0, '마포대교'),
 (100851174.0, '행주대교'),
 (99471344.0, '잠실대교'),
 (89907701.0, '한강대교'),
 (78150029.0, '잠수교'),
 (75898176.0, '동호대교'),
 (70572787.0, '천호대교'),
 (66286549.0, '올림픽대교'),
 (57730664.0, '동작대교'),
 (49674940.0, '원효대교'),
 (45418028.0, '서강대교'),
 (16357856.0, '광진교')]